In [129]:
import random
import math
import matplotlib.pyplot as plt
import time
import numpy as np
from point import Point
import json

# Load the Layout Pool from Static Generation

In [130]:
with open("data/static_layout_pool.json", 'r') as openfile:
    json_object = json.load(openfile)

WIDTH = json_object["width"]
LENGTH = json_object["length"]
layouts = [[Point(p[0], p[1]) for p in lay] for lay in json_object["layouts"]]

# Dynamic Trial Shuffling
- We shuffle the N trials such that the boundary visibility alternates each time.
- We read N/2 trials from static generation.
- Boundary visibility is alternating, so we simply assume border_visibility = i % 2

## Generation

- At least K = 5 trials between trials of the same pairing.
- We make sure that between adjacent pairs, the ending location and starting location is always opposite.
    - Also, make sure these distances obey MIN_DIST_PATH

In [131]:
NUM_SHUFFLES = 50
DIST_BETWEEN_COPY = 5 # K
MIN_DIST_PATH = 1.5
LAYOUT_COUNT = len(layouts)

In [134]:
def generate_random_shuffle():
    # we interleave to random lists of 1...n/2, therefore it is guaranteed that the random shuffling
    #is valid for the constraint: boundary visibility alternates each time. each layout gets exactly 
    #one case with border, exactly one without border
    def random_half_shuffle():
        l = list(range(len(layouts)))
        random.shuffle(l)
        return l
    
    l = [x for t in zip(random_half_shuffle(), random_half_shuffle()) for x in t]
    return l

def is_valid_shuffle(order):
    # At least K = 5 trials between trials of the same pairing.
    # since we generate this as half and half, we only need to check the first and last K elements
    K = DIST_BETWEEN_COPY
    for i in range(K, len(order)):
        if order[i] in order[i-K:i]:
            #print("----", i, K, order[i], order[i-K:i])
            return False
    
    # each pair gets one occurence with border, one occurence without border
    # as a property of generate_random_shuffle, therefore we do not test for it.
    
    #TODO: We make sure that between adjacent pairs, the ending location and starting location is always opposite.
    #TODO: Also, make sure these distances obey MIN_DIST_PATH
    
    rule_adjacentPairNotSameSide_violations = 0
    rule_minDistPath_violations = 0
    mirrored_y_for_rule = [False] # first layout is the anchor, never mirrored
    carrying_mirror_y = False
    
    for i in range(len(order) - 1):
        pre_point = layouts[order[i]][-1]
        post_point = layouts[order[i+1]][0]

        pre_side = pre_point.y < LENGTH/2
        post_side = post_point.y < LENGTH/2
        
        mirrored_pre_point = Point(pre_point.x, LENGTH - pre_point.y)
        mirrored_post_point = Point(post_point.x, LENGTH - post_point.y)
        
        # if there is any violation, try to mirror this next trial
        side_violation = pre_side == post_side
        dist_violation = (mirrored_pre_point if carrying_mirror_y else pre_point).dist(post_point) < MIN_DIST_PATH
        
        if (not side_violation) and dist_violation:
            return False # this will be impossible to solve, until, perhaps, x-mirroring is implemented
        
        if side_violation or dist_violation:
            # mirror across the border so that (hopefully) this violation won't happen
            carrying_mirror_y = not carrying_mirror_y
            side_violation = not side_violation # by definition, changing the mirroring causes a side violation
            
        mirrored_y_for_rule.append(carrying_mirror_y)
        
        # check rules again
        pre_for_checks = mirrored_pre_point if mirrored_y_for_rule[-2] else pre_point
        post_for_checks = mirrored_post_point if mirrored_y_for_rule[-1] else post_point
        
        
        dist_violation = pre_for_checks.dist(post_for_checks) < MIN_DIST_PATH
        
        if side_violation:
            return False
        if dist_violation:
            return False

    return rule_adjacentPairNotSameSide_violations + rule_minDistPath_violations == 0
        
def generate_valid_shuffle():
    shuffle = generate_random_shuffle()
    tries = 1
    while not is_valid_shuffle(shuffle):
        shuffle = generate_random_shuffle()
        tries += 1
    print(f"====== {tries} =====")
    return shuffle

In [145]:
shuffles = [generate_valid_shuffle() for i in range(25)]
print("\n".join([".".join([str(c) for c in shuf])[:10]  for shuf in shuffles]))

0
====== 47563 =====
0
====== 13484 =====
0
====== 67749 =====
0
====== 19543 =====
0
====== 61782 =====
0
====== 87498 =====
0
====== 11506 =====
0
====== 61801 =====
0
====== 227879 =====
0
====== 21701 =====
0
====== 181809 =====
0
====== 123734 =====
0
====== 37537 =====
0
====== 17459 =====
0
====== 14218 =====
0
====== 61082 =====
0
====== 35978 =====
0
====== 545 =====
0
====== 19562 =====
0
====== 145601 =====
0
====== 73249 =====
0
====== 4632 =====
0
====== 50759 =====
0
====== 41871 =====
0
====== 115367 =====
19.25.14.4
3.0.32.6.6
35.20.6.28
0.24.33.7.
28.24.9.36
29.12.26.3
3.23.36.17
21.31.14.3
8.2.10.10.
36.22.24.3
3.13.17.36
24.0.34.11
30.12.11.2
5.1.26.12.
36.28.11.1
23.32.7.6.
11.19.37.1
5.21.38.5.
20.9.7.16.
0.14.13.23
35.34.0.25
29.3.7.37.
5.28.19.39
13.37.16.2
22.37.13.2


## Checks

- We make sure that apmprox. half the pairs happen in A->B, and the other half happens in B->A
- We want to check the number of objects within the same side across a sliding window of 5 trials.

# Objects
- 5 different shapes (circle, cube, cylinder, pyramid, star)
- 5 different colors from a color-blind-friendly palette.
-> 25 different combinations.

Any object generated must not have been in the last 10 shown object.